In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# User-based recommendations

In [ ]:
# the file is in a Box folder 
ratings = pd.read_csv('ratings_large.csv')
ratings

In [ ]:
# (transpose) ratings matrix
ratings_matrix = ratings.pivot('title','userId')['rating']
ratings_matrix

In [ ]:
# select a userId 
user = 4
user_ratings = ratings_matrix.loc[:,user]
user_ratings

In [ ]:
# drop user column from the ratings matrix
ratings_matrix.drop(user,axis=1,inplace=True)

In [ ]:
# movies rated by the user
user_ratings[user_ratings.notna()].index.to_list()

In [ ]:
# movies not rated by the user
user_ratings[user_ratings.isna()].index.to_list()

In [ ]:
# ratings distribution
user_ratings.value_counts(dropna=False)

In [ ]:
# user top movies
user_ratings.sort_values(ascending=False).head(20)

In [ ]:
# user bottom movies
user_ratings.sort_values().head(20)

In [ ]:
# user mean rating
user_mean = user_ratings.mean()
user_mean

In [ ]:
# user standard deviation rating
user_std = user_ratings.std()
user_std

## The z-scores matrix

In [ ]:
# matrix of z-scores (scale-free ratings)
z_scores = (ratings_matrix - ratings_matrix.mean())/ratings_matrix.std()
z_scores

## Similarities

In [ ]:
# user similarity function = correlation 
similarities = ratings_matrix.corrwith(user_ratings)
similarities

## Prediction function

In [ ]:
# select number of neighbors
k = 20

# select an (unrated) item: 'Dawn of the Dead (2004)', "Zoolander (2001)", 'Duck Soup (1933)'
item = 'Duck Soup (1933)'

In [ ]:
# k nearest neighbors similarities
knn_sim = similarities[ratings_matrix.loc[item].notna()].sort_values(ascending=False).head(k)
knn_sim

In [ ]:
# normalization factor
total = knn_sim.abs().sum()
total

In [ ]:
# k nearest neighbors
knn = knn_sim.index
knn

In [ ]:
knn_z_scores = z_scores.loc[item,knn]
knn_z_scores

In [ ]:
# prediction
prediction = user_mean + user_std*knn_z_scores.dot(knn_sim)/total
print(item+': '+str(np.round(prediction,1)))

In [ ]:
# the same in one cell:

# select number of neighbors
k = 20

# select item
item = "Zoolander (2001)"# 

# k nearest neighbors similarities
knn_sim = similarities[ratings_matrix.loc[item].notna()].sort_values(ascending=False).head(k)
    
# normalization factor
total = knn_sim.abs().sum()

# k nearest neighbors
knn = knn_sim.index

# k nearest neighbors z-scores
knn_z_scores = z_scores.loc[item,knn]

# prediction
prediction = user_mean + user_std*knn_sim.dot(knn_z_scores)/total

print(item+': '+str(np.round(prediction,1)))

## Fake user

In [ ]:
# fake user likes sci-fi films, action-adventures films, and doesn't like romance films, kid movies
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Theory/master/Data/fake_user.csv'
user = pd.read_csv(url,index_col='title', squeeze=True)
user

In [ ]:
user_mean = user.mean()
user_std = user.std()

In [ ]:
# user similarities
similarities = ratings_matrix.corrwith(user)

In [ ]:
# not rated movies
not_rated_movies = user[user.isna()].index.to_list()
not_rated_movies

In [ ]:
k = 20
for item in not_rated_movies:
    # k nearest neighbors similarities
    knn_sim = similarities[ratings_matrix.loc[item].notna()].sort_values(ascending=False).head(k)

    # normalization factor
    total = knn_sim.abs().sum()
    
    # k nearest neighbors
    knn = knn_sim.index
    
    # k nearest neighbors z-scores
    knn_z_scores = z_scores.loc[item,knn]
    

    # prediction
    prediction = user_mean + user_std*knn_sim.dot(knn_z_scores)/total
    user.loc[item] = prediction

In [ ]:
# top 20 recommendations
user.loc[not_rated_movies].sort_values(ascending=False).head(20)

In [ ]:
# bottom 20 recommendations
user.loc[not_rated_movies].sort_values(ascending=False).tail(20)

## The impact of the long tail

In [ ]:
# fake user likes sci-fi films, action-adventures films, and doesn't like romance films, kid movies
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Theory/master/Data/fake_user.csv'
user = pd.read_csv(url,index_col='title', squeeze=True)
user

In [ ]:
# number of ratings each movie received
n_ratings = ratings.groupby('title')['rating'].count()
n_ratings

In [ ]:
n_ratings.plot.hist(bins=20)
plt.xlabel('number of ratings')
plt.ylabel('number of movies')

In [ ]:
# popular movies
n_ratings[n_ratings>20000].sort_values(ascending=False)

In [ ]:
# number of users
n_users = ratings['userId'].nunique()
n_users

In [ ]:
# weights
weights = np.log(n_users/n_ratings)
weights

In [ ]:
weights['Forrest Gump (1994)']

In [ ]:
# pandas does not have a built-in function that computes weighted correlations
def weighted_corrwith(y):
    
    x = user
    
    # means
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    
    # denominator
    den1 = np.sqrt(np.sum((weights*(x-mean_x)**2)))
    den2 = np.sqrt(np.sum((weights*(y-mean_y)**2)))
    den = den1*den2
    
    # numerator
    num = np.sum(weights*(x-mean_x)*(y-mean_y))
    
    return num/den

In [ ]:
similarities = ratings_matrix.apply(weighted_corrwith)
similarities

In [ ]:
# not rated movies
not_rated_movies = user[user.isna()].index.to_list()
not_rated_movies

In [ ]:
k = 20
for item in not_rated_movies:
    # k nearest neighbors similarities
    knn_sim = similarities[ratings_matrix.loc[item].notna()].sort_values(ascending=False).head(k)

    # normalization factor
    total = knn_sim.abs().sum()
    
    # k nearest neighbors
    knn = knn_sim.index
    
    # k nearest neighbors z-scores
    knn_z_scores = z_scores.loc[item,knn]
    

    # prediction
    prediction = user_mean + user_std*knn_sim.dot(knn_z_scores)/total
    user.loc[item] = prediction

In [ ]:
# top 20 recommendations
user.loc[not_rated_movies].sort_values(ascending=False).head(20)

## Appendix: create a fake user

In [ ]:
fake_user = pd.DataFrame(index=ratings_matrix.index,columns=['rating'])
fake_user

In [ ]:
fake_user.to_csv('fake_kid_user.csv')